In [10]:
from utils import extract_data, load_data
import os
import pickle
import pandas as pd
from IPython.display import display

In [11]:
def extract(model: str):
    raw_data = load_data(os.path.join('data', 'raw', f'bonbanh.{model}.pkl'))
    clean_data = []
    for e in raw_data:
        clean_data.append(extract_data(e))
    df = pd.DataFrame(clean_data)
    display(df.head())
    df.to_pickle(os.path.join('data', 'clean', f'bonbanh.{model}.pkl'))

In [12]:
# extract('audi')
# extract('bentley')
# extract('bmw')
# extract('chevrolet')
# extract('daewoo')
# extract('ford')
# extract('honda')
# extract('hyundai')
# extract('isuzu')
# extract('jeep')
# extract('kia')
# extract('landrover')
# extract('lexus')
# extract('mazda')
# extract('mercedes_benz')
# extract('mg')
# extract('mini')
# extract('mitsubishi')
# extract('nissan')
# extract('peugeot')
# extract('porsche')
# extract('subaru')
# extract('suzuki')
# extract('toyota')
# extract('vinfast')
# extract('volkswagen')
# extract('volvo')

In [13]:
def merge_data(folder_path):
    data_df = pd.DataFrame()
    files = os.listdir(folder_path)
    sorted_files = sorted(files)

    for file_name in sorted_files:
        if file_name.startswith('bonbanh'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'rb') as f:
                content = pickle.load(f)
                if isinstance(content, pd.DataFrame):
                    data_df = pd.concat([data_df, content], ignore_index=True)
                    print(f"{file_name}: {content.shape[0]}")

    data_df.drop(columns=["Mô tả"], axis=1, inplace=True)
    return data_df

In [14]:
FOLDER_PATH = os.path.join("data", "clean")
cars = merge_data(FOLDER_PATH)

bonbanh.audi.pkl: 315
bonbanh.bentley.pkl: 65
bonbanh.bmw.pkl: 682
bonbanh.chevrolet.pkl: 554
bonbanh.daewoo.pkl: 248
bonbanh.ford.pkl: 3155
bonbanh.honda.pkl: 1352
bonbanh.hyundai.pkl: 3475
bonbanh.isuzu.pkl: 120
bonbanh.jeep.pkl: 89
bonbanh.kia.pkl: 2900
bonbanh.landrover.pkl: 411
bonbanh.lexus.pkl: 1177
bonbanh.mazda.pkl: 2147
bonbanh.mercedes_benz.pkl: 3329
bonbanh.mg.pkl: 376
bonbanh.mini.pkl: 113
bonbanh.mitsubishi.pkl: 1474
bonbanh.nissan.pkl: 300
bonbanh.peugeot.pkl: 275
bonbanh.porsche.pkl: 429
bonbanh.subaru.pkl: 212
bonbanh.suzuki.pkl: 343
bonbanh.toyota.pkl: 5860
bonbanh.vinfast.pkl: 969
bonbanh.volkswagen.pkl: 361
bonbanh.volvo.pkl: 150


In [15]:
cars.drop_duplicates(subset=["Mã xe"], inplace=True)
cars["Số Km đã đi"] = cars["Số Km đã đi"].str.replace(" Km", "").str.replace(",", "")
cars["Số Km đã đi"] = cars["Số Km đã đi"].fillna(0)
cars = cars[cars["Dẫn động"] != "-"]
cars["Dẫn động"] = cars["Dẫn động"].apply(lambda x: str(x).split()[0])
cars["Loại động cơ"] = cars["Động cơ"].apply(lambda x: str(x).split()[0])
cars["Dung tích động cơ"] = cars["Động cơ"].apply(
    lambda x: float(x.split(" ")[-2]) if len(x.split(" ")) > 1 else None
)
cars.dropna(subset=["Dung tích động cơ"], inplace=True)
cars["Tình trạng"] = cars["Tình trạng"].apply(lambda x: "cũ" if (len(str(x).split()) == 3) else "mới")
cars["Xuất xứ"] = cars["Xuất xứ"].replace("Lắp ráp trong nước", "Trong nước")

In [16]:
def cut_year(name):
    if "-" in name:
        name = name.rsplit("-", 1)[0]
    return name.strip()


cars["Tên xe"] = cars["Tên xe"].apply(cut_year)
cars["Tên hãng xe"] = cars["Tên xe"].apply(lambda x: x.split(" ")[0])
cars["Tên dòng xe"] = cars["Tên xe"].apply(lambda x: " ".join(x.split(" ")[1:]))
cars.drop(columns=["Tình trạng chung", "Động cơ", "Tên xe"], inplace=True)

columns = [
    "Mã xe", "URL", "Tên hãng xe", "Tên dòng xe", "Năm sản xuất",
    "Tỉnh", "Tình trạng", "Số Km đã đi", "Xuất xứ",
    "Kiểu dáng", "Hộp số", "Màu ngoại thất", "Màu nội thất",
    "Số chỗ ngồi", "Số cửa", "Dẫn động", "Loại động cơ", "Dung tích động cơ", "Giá"
]
cars = cars.reindex(columns=columns)

cars["Số chỗ ngồi"] = cars["Số chỗ ngồi"].str.split(" ").str[0].astype(int)
cars["Số cửa"] = cars["Số cửa"].str.split(" ").str[0].astype(int)
cars["Số Km đã đi"] = cars["Số Km đã đi"].astype(int)
cars["Năm sản xuất"] = cars["Năm sản xuất"].astype(int)
cars["Giá"] = cars["Giá"].astype(int) / 1000000
cars["Giá"] = cars["Giá"].astype(float)

In [17]:
print(cars.shape)
cars.head()

(30191, 19)


,Mã xe,URL,Tên hãng xe,Tên dòng xe,Năm sản xuất,Tỉnh,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Loại động cơ,Dung tích động cơ,Giá
0,5596529,https://bonbanh.com/xe-audi-a8-l-55-tfsi-quatt...,Audi,A8 L 55 TFSI Quattro,2020,TP HCM,cũ,30000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,AWD,Xăng,3.0,2950.0
1,5555194,https://bonbanh.com/xe-audi-a6-2.0-tfsi-2014-5...,Audi,A6 2.0 TFSI,2014,Hà Nội,cũ,94000,Nhập khẩu,Sedan,Số tự động,Xanh,Nâu,5,4,FWD,Xăng,2.0,560.0
2,5464113,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2016-5...,Audi,A4 2.0 TFSI,2016,TP HCM,cũ,50000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,FWD,Xăng,2.0,690.0
3,5532313,https://bonbanh.com/xe-audi-q3-2.0-quattro-201...,Audi,Q3 2.0 Quattro,2016,Hà Nội,cũ,76000,Nhập khẩu,Crossover,Số tự động,Xanh,Đen,5,5,AWD,Xăng,2.0,699.0
4,5346160,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2017-5...,Audi,A4 2.0 TFSI,2017,TP HCM,cũ,0,Nhập khẩu,Sedan,Số tự động,Đen,Nâu,5,4,FWD,Xăng,2.0,799.0


In [18]:
cars.to_pickle(os.path.join("data", "clean", "cars.pkl"))
cars.to_csv(os.path.join("data", "clean", "cars.csv"))